In [1]:
!nvidia-smi

Tue Apr  9 12:52:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.73       Driver Version: 410.73       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 760     Off  | 00000000:01:00.0 N/A |                  N/A |
| 38%   42C    P8    N/A /  N/A |     26MiB /  4034MiB |     N/A      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 760     Off  | 00000000:06:00.0 N/A |                  N/A |
| 38%   41C    P8    N/A /  N/A |      1MiB /  4037MiB |     N/A      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
# %load cnn_regression-Copy1.py
# USAGE
# python cnn_regression.py --dataset Houses-dataset/Houses\ Dataset/

# import the necessary packages
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from pyimagesearch import models
import numpy as np
import argparse
import locale
import os
import sys
from matplotlib.image import imread
import matplotlib.pyplot as plt
import tensorflow as tf


IMG_WIDTH = 1400
IMG_HEIGHT = 1400
ROW_SLICE = slice(0, 1400)
COL_SLICE = slice(1000, None)


def cut_X(arr, reshape = None):
    x_cut = arr[:,960:1300,600:]
    if reshape:
        if len(x_cut.shape)>3:
            x_cut = x_cut[...,0]
            x_cut_out = x_cut.reshape(x_cut.shape[0],x_cut.shape[1]*x_cut.shape[2])
    else:
        x_cut_out = x_cut
    return x_cut_out


#data_dir = "/storage/yw18581/data/"
data_dir = "/data/uob"
TRAIN_VAL_TEST_DIR = os.path.join(data_dir, "train_validation_test")

N_EPOCHS=200

CHECKPOINT_FOLDER_PATH = os.path.join(data_dir, 'trained_models')
TASK_NAME = 'CNN_regression_pyimage_{}epochs'.format(N_EPOCHS)
TASK_FOLDER_PATH = os.path.join(CHECKPOINT_FOLDER_PATH, TASK_NAME)

if not os.path.exists(TASK_FOLDER_PATH):
    os.makedirs(TASK_FOLDER_PATH)


X_train = np.load(os.path.join(TRAIN_VAL_TEST_DIR,"Xy_train_dist.npz"))["y"]
X_val = np.load(os.path.join(TRAIN_VAL_TEST_DIR,"Xy_val_dist.npz"))["y"]
X_test = np.load(os.path.join(TRAIN_VAL_TEST_DIR,"Xy_test_dist.npz"))["y"]

y_train = np.load(os.path.join(TRAIN_VAL_TEST_DIR,"Xy_train_dist.npz"))["dist"]
y_val = np.load(os.path.join(TRAIN_VAL_TEST_DIR,"Xy_val_dist.npz"))["dist"]
y_test = np.load(os.path.join(TRAIN_VAL_TEST_DIR,"Xy_test_dist.npz"))["dist"]

X_train_cut = cut_X(X_train)
X_val_cut = cut_X(X_val)
X_test_cut = cut_X(X_test)


# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxDist = np.max(y_train)
print(maxDist)
trainY = y_train/maxDist
valY = y_val/maxDist
testY = y_test/maxDist

# create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# price *predictions* and the *actual prices*

_, width, height, depth,  = X_train_cut.shape

model = models.create_cnn(height,width, depth, regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / N_EPOCHS)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
model.summary()

# train the model
print("[INFO] training model...")
model.fit(X_train_cut, trainY, validation_data=(X_val_cut, valY),
	epochs=N_EPOCHS, batch_size=8, verbose=1)




Using TensorFlow backend.


25
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 340, 800, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 340, 800, 16)      160       
_________________________________________________________________
activation_1 (Activation)    (None, 340, 800, 16)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 340, 800, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 170, 400, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 170, 400, 32)      4640      
_________________________________________________________________
activation_2 (Activation)    (None, 170, 400, 32)      0         
_______

768/768 [==============================] - 19s 25ms/step - loss: 19.0945 - val_loss: 10.6894
Epoch 49/200
768/768 [==============================] - 19s 25ms/step - loss: 19.3432 - val_loss: 7.8791
Epoch 50/200
768/768 [==============================] - 19s 25ms/step - loss: 17.7434 - val_loss: 5.7329
Epoch 51/200
768/768 [==============================] - 19s 25ms/step - loss: 20.6159 - val_loss: 9.2945
Epoch 52/200
768/768 [==============================] - 19s 25ms/step - loss: 20.0886 - val_loss: 11.1366
Epoch 53/200
768/768 [==============================] - 19s 25ms/step - loss: 18.7155 - val_loss: 9.7387
Epoch 54/200
768/768 [==============================] - 19s 25ms/step - loss: 17.5865 - val_loss: 8.7909
Epoch 55/200
768/768 [==============================] - 19s 25ms/step - loss: 17.2325 - val_loss: 10.3860
Epoch 56/200
768/768 [==============================] - 19s 25ms/step - loss: 18.0866 - val_loss: 10.5222
Epoch 57/200
768/768 [==============================] - 19s 25ms

768/768 [==============================] - 19s 25ms/step - loss: 16.6700 - val_loss: 8.5147
Epoch 126/200
768/768 [==============================] - 19s 25ms/step - loss: 15.9904 - val_loss: 9.1570
Epoch 127/200
768/768 [==============================] - 19s 25ms/step - loss: 17.7788 - val_loss: 5.3726
Epoch 128/200
768/768 [==============================] - 19s 25ms/step - loss: 16.5308 - val_loss: 8.4789
Epoch 129/200
768/768 [==============================] - 19s 25ms/step - loss: 16.2642 - val_loss: 10.6526
Epoch 130/200
768/768 [==============================] - 19s 25ms/step - loss: 17.4110 - val_loss: 10.7447
Epoch 131/200
768/768 [==============================] - 19s 25ms/step - loss: 16.4375 - val_loss: 8.2729
Epoch 132/200
768/768 [==============================] - 19s 25ms/step - loss: 15.8864 - val_loss: 11.7584
Epoch 133/200
768/768 [==============================] - 19s 25ms/step - loss: 16.6237 - val_loss: 8.0502
Epoch 134/200
768/768 [==============================] - 

NameError: name 'df' is not defined

In [4]:
# make predictions on the testing data
print("[INFO] predicting distances...")
preds = model.predict(X_test_cut)

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)



[INFO] predicting distances...


In [6]:
mean

9.960592955661317

In [7]:
from sklearn.metrics import mean_squared_error

In [8]:
mean_squared_error(preds, y_test)

168.8568395557665

In [ ]:

# compute the mean and standard deviation of the absolute percentage
# difference

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))